In [1]:
from sagas.ofbiz.entities import OfEntity as e, oc, MetaEntity, all_entities
e('meta').Person

,internal,name,primary,type,field type
0,,partyId,*,id,id
1,,salutation,,name,name
2,,firstName,,name,name
3,,middleName,,name,name
4,,lastName,,name,name
5,,personalTitle,,name,name
6,,suffix,,name,name
7,,nickname,,name,name
8,,firstNameLocal,,name,name
9,,middleNameLocal,,name,name


In [2]:
ent=MetaEntity('PartyNameContactMechView')
print(ent.model.getPlainTableName() is None or len(ent.model.getPlainTableName())==0)
ent=MetaEntity('Party')
print(ent.model.getPlainTableName())

True
PARTY


In [6]:
oc.import_package('com.sagas.generic.*')
print(oc.j.Utils.isViewEntity(ent.model))
print(oc.j.Utils.isViewEntity(MetaEntity('PartyNameContactMechView').model))

False
True


In [7]:
def build_field_index(ents):
    field_index={}
    for ent_name in ents:
        ent=MetaEntity(ent_name)        
        if not oc.j.Utils.isViewEntity(ent.model):
            for fld in ent.model.getFieldsIterator():
                if not fld.getIsAutoCreatedInternal():
                    fld_name=fld.getName()
                    if fld_name not in field_index:
                        field_index[fld_name]=[ent_name]
                    else:
                        field_index[fld_name].append(ent_name)
    return field_index

ents=['Person', 'Party']
build_field_index(ents)

{'birthDate': ['Person'],
 'cardId': ['Person'],
 'comments': ['Person'],
 'createdByUserLogin': ['Party'],
 'createdDate': ['Party'],
 'dataSourceId': ['Party'],
 'deceasedDate': ['Person'],
 'description': ['Party'],
 'employmentStatusEnumId': ['Person'],
 'existingCustomer': ['Person'],
 'externalId': ['Party'],
 'firstName': ['Person'],
 'firstNameLocal': ['Person'],
 'gender': ['Person'],
 'height': ['Person'],
 'isUnread': ['Party'],
 'lastModifiedByUserLogin': ['Party'],
 'lastModifiedDate': ['Party'],
 'lastName': ['Person'],
 'lastNameLocal': ['Person'],
 'maritalStatus': ['Person'],
 'memberId': ['Person'],
 'middleName': ['Person'],
 'middleNameLocal': ['Person'],
 'monthsWithEmployer': ['Person'],
 'mothersMaidenName': ['Person'],
 'nickname': ['Person'],
 'occupation': ['Person'],
 'otherLocal': ['Person'],
 'partyId': ['Person', 'Party'],
 'partyTypeId': ['Party'],
 'passportExpireDate': ['Person'],
 'passportNumber': ['Person'],
 'personalTitle': ['Person'],
 'preferredC

In [8]:
ents=all_entities()
idx=build_field_index(ents)
print(len(idx))

2216


In [10]:
import resources_pb2 as res
import protobuf_utils
idx_b={}
for k,v in idx.items():
    idx_b[k]=res.RsEntityReference(entities=v)
rs=res.RsEntities(fieldRefs=idx_b)
protobuf_utils.write_proto_to(rs, 'data/resources/entities_index.data')

print(len(rs.fieldRefs))

2216


In [17]:
target='data/resources/entities_index.data'
rs=res.RsEntities()
protobuf_utils.read_proto(rs, target)

print(len(rs.fieldRefs))
print(rs.fieldRefs['lastName'])

2216
entities: "Person"
entities: "OfbizDemo"
entities: "PartyNameHistory"



In [13]:
def is_field(qname):
    return qname in rs.fieldRefs

print(is_field('lastName'))
print(is_field('fakeName'))

True
False


In [3]:
from sagas.ofbiz.resources import read_resource
resource, rs_lookups = read_resource()

def is_description(word, lang='zh'):
    lang_idx = rs_lookups.indexTable[lang]

    if word in lang_idx.indexes:
        keys = lang_idx.indexes[word]
        for key in keys.value:
            # print(word, "☞", key)
            if '.description.' in key:
                return True
    return False

lang='zh'
word='会员'
is_description(word, lang)

True